In [1]:

import math
import os
import shutil
from collections import Counter

def reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,
                   valid_ratio):
    # 读取训练数据标签。
    with open(os.path.join(data_dir, label_file), 'r') as f:
        # 跳过文件头行（栏名称）。
        lines = f.readlines()[1:]
        tokens = [l.rstrip().split(',') for l in lines]
        idx_label = dict(((idx, label) for idx, label in tokens))
    labels = set(idx_label.values())

    num_train = len(os.listdir(os.path.join(data_dir, train_dir)))
    # 训练集中数量最少一类的狗的数量。
    min_num_train_per_label = (
        Counter(idx_label.values()).most_common()[:-2:-1][0][1])
    # 验证集中每类狗的数量。
    num_valid_per_label = math.floor(min_num_train_per_label * valid_ratio)
    label_count = dict()

    def mkdir_if_not_exist(path):
        if not os.path.exists(os.path.join(*path)):
            os.makedirs(os.path.join(*path))

    # 整理训练和验证集。
    for train_file in os.listdir(os.path.join(data_dir, train_dir)):
        idx = train_file.split('.')[0]
        label = idx_label[idx]
        mkdir_if_not_exist([data_dir, input_dir, 'train_valid', label])
        shutil.copy(os.path.join(data_dir, train_dir, train_file),
                    os.path.join(data_dir, input_dir, 'train_valid', label))
        if label not in label_count or label_count[label] < num_valid_per_label:
            mkdir_if_not_exist([data_dir, input_dir, 'valid', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            mkdir_if_not_exist([data_dir, input_dir, 'train', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'train', label))

    # 整理测试集。
    mkdir_if_not_exist([data_dir, input_dir, 'test', 'unknown'])
    for test_file in os.listdir(os.path.join(data_dir, test_dir)):
        shutil.copy(os.path.join(data_dir, test_dir, test_file),
                    os.path.join(data_dir, input_dir, 'test', 'unknown'))



In [12]:
data_dir = u"/media/yijie/文档/dataset/ImageNetDog"
label_file = 'labels.csv'
train_dir = 'train'
test_dir = 'test'
input_dir = 'train_valid_test'
batch_size = 80
valid_ratio = 0.1
#reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,
 #                  valid_ratio)

In [13]:
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
import numpy as np

def transform_train(data, label):
    im = image.imresize(data.astype('float32') / 255, 96, 96)
    auglist = image.CreateAugmenter(data_shape=(3, 96, 96), resize=0,
                        rand_crop=False, rand_resize=False, rand_mirror=True,
                        mean=None, std=None,
                        brightness=0, contrast=0,
                        saturation=0, hue=0,
                        pca_noise=0, rand_gray=0, inter_method=2)
    for aug in auglist:
        im = aug(im)
    # 将数据格式从"高*宽*通道"改为"通道*高*宽"。
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

def transform_test(data, label):
    im = image.imresize(data.astype('float32') / 255, 96, 96)
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

In [11]:
input_str = "/media/yijie/文档/dataset/ImageNetDog/train_valid_test/"

# 读取原始图像文件。flag=1说明输入图像有三个通道（彩色）。
train_ds = vision.ImageFolderDataset(input_str + 'train', flag=1,
                                     transform=transform_train)
valid_ds = vision.ImageFolderDataset(input_str + 'valid', flag=1,
                                     transform=transform_test)
train_valid_ds = vision.ImageFolderDataset(input_str + 'train_valid',
                                           flag=1, transform=transform_train)
test_ds = vision.ImageFolderDataset(input_str + 'test', flag=1,
                                     transform=transform_test)

loader = gluon.data.DataLoader
train_data = loader(train_ds, batch_size, shuffle=True, last_batch='keep')
valid_data = loader(valid_ds, batch_size, shuffle=True, last_batch='keep')
train_valid_data = loader(train_valid_ds, batch_size, shuffle=True,
                          last_batch='keep')
test_data = loader(test_ds, batch_size, shuffle=False, last_batch='keep')

# 交叉熵损失函数。
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()


AttributeError: 'ImageFolderDataset' object has no attribute 'shape'

In [8]:
from mxnet.gluon import nn
from mxnet import nd

class Residual(nn.HybridBlock):
    def __init__(self, channels, same_shape=True, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.same_shape = same_shape
        with self.name_scope():
            strides = 1 if same_shape else 2
            self.conv1 = nn.Conv2D(channels, kernel_size=3, padding=1,
                                  strides=strides)
            self.bn1 = nn.BatchNorm()
            self.conv2 = nn.Conv2D(channels, kernel_size=3, padding=1)
            self.bn2 = nn.BatchNorm()
            if not same_shape:
                self.conv3 = nn.Conv2D(channels, kernel_size=1,
                                      strides=strides)

    def hybrid_forward(self, F, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(out + x)


class ResNet(nn.HybridBlock):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super(ResNet, self).__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            net = self.net = nn.HybridSequential()
            # 模块1
            net.add(nn.Conv2D(channels=32, kernel_size=3, strides=1,
                              padding=1))
            net.add(nn.BatchNorm())
            net.add(nn.Activation(activation='relu'))
            # 模块2
            for _ in range(3):
                net.add(Residual(channels=32))
            # 模块3
            net.add(Residual(channels=64, same_shape=False))
            for _ in range(2):
                net.add(Residual(channels=64))
            # 模块4
            net.add(Residual(channels=128, same_shape=False))
            for _ in range(2):
                net.add(Residual(channels=128))
            # 模块5
            net.add(nn.GlobalAvgPool2D())
            net.add(nn.Flatten())
            net.add(nn.Dense(num_classes))

    def hybrid_forward(self, F, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print('Block %d output: %s'%(i+1, out.shape))
        return out
def get_net(ctx):
    num_outputs = 120
    net = ResNet(num_outputs)
    net.initialize(ctx=ctx, init=init.Xavier())
    return net


In [1]:
import datetime
import sys
sys.path.append('..')
import utils
from matplotlib import pyplot as plt

def get_loss(data, net, ctx):
    loss = 0.0
    for feas, label in data:
        label = label.as_in_context(ctx)
        output = net(feas.as_in_context(ctx))
        cross_entropy = softmax_cross_entropy(output, label)
        loss += nd.mean(cross_entropy).asscalar()
    return loss / len(data)

def train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_period,
          lr_decay):
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9,
                                      'wd': wd})
    prev_time = datetime.datetime.now()
    plt_train_loss = []
    plt_valid_loss = []

    for epoch in range(num_epochs):
        train_loss = 0.0
        if epoch < 81 and epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        if epoch > 81 and epoch % 10 == 0:
            trainer.set_learning_rate(trainer.learning_rate * 0.4)
        for data, label in train_data:
            label = label.as_in_context(ctx)
            with autograd.record():
                output = net(data.as_in_context(ctx))
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            train_loss += nd.mean(loss).asscalar()
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_loss = get_loss(valid_data, net, ctx)
            epoch_str = ("Epoch %d. Train loss: %f, Valid loss %f, "
                         % (epoch, train_loss / len(train_data), valid_loss))
            plt_train_loss.append(train_loss / len(train_data))
            plt_valid_loss.append(valid_loss)
        else:
            epoch_str = ("Epoch %d. Train loss: %f, "
                         % (epoch, train_loss / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))

    # plot 
    if valid_data is not None:
        plt.plot(plt_train_loss)
        plt.plot(plt_valid_loss)
        plt.legend(['train_loss','test_loss'])
        plt.savefig("Loss22.png")


In [10]:
for data, label in train_data:
    print label


[ 107.   73.   35.    8.   31.   74.   75.   87.    4.    6.   17.   97.
   17.   65.   17.   12.   98.   90.   73.   13.   80.  101.   20.   24.
  101.   15.  115.   56.  104.  117.   43.   56.   85.   81.   13.   97.
   15.   30.   38.   73.   85.   97.  113.   84.   60.   25.  111.   86.
   22.   86.   78.  100.   40.   61.   37.   81.   10.   79.   86.   82.
   52.    0.   68.  117.   94.    5.   63.   60.   16.  105.   15.   67.
   97.   20.   10.    5.  119.   69.   73.   61.]
<NDArray 80 @cpu(0)>

[  13.   48.    3.    8.   70.   88.   44.   42.   91.   23.   36.   11.
   88.   95.   40.  110.   12.   55.   45.  116.   61.   33.   79.   70.
   41.   17.   71.   10.   53.   97.   58.   63.   83.   59.   86.   42.
   54.   68.   86.   72.   69.   16.   86.   85.   54.    6.   67.   59.
  118.  109.   86.   87.   59.   71.   44.   84.   59.  100.  101.   20.
   21.   82.   27.    1.   47.    0.   84.   99.   95.   13.  115.   52.
   64.   85.   60.   74.   17.  111.   52.   76.]
<

KeyboardInterrupt: 

In [14]:
ctx = utils.try_gpu()
num_epochs = 100
learning_rate = 0.01
weight_decay = 5e-4
lr_period = 80
lr_decay = 0.1

net = get_net(ctx)
net.hybridize()
train(net, train_data, valid_data, num_epochs, learning_rate,
      weight_decay, ctx, lr_period, lr_decay)

Epoch 0. Train loss: 5.001737, Valid loss 4.842694, Time 00:01:49, lr 0.001
Epoch 1. Train loss: 4.741746, Valid loss 4.761680, Time 00:01:54, lr 0.001
Epoch 2. Train loss: 4.662718, Valid loss 4.742021, Time 00:01:54, lr 0.001
Epoch 3. Train loss: 4.610438, Valid loss 4.665113, Time 00:01:54, lr 0.001
Epoch 4. Train loss: 4.565544, Valid loss 4.606892, Time 00:01:54, lr 0.001
Epoch 5. Train loss: 4.524102, Valid loss 4.622794, Time 00:01:54, lr 0.001
Epoch 6. Train loss: 4.486539, Valid loss 4.523738, Time 00:01:54, lr 0.001
Epoch 7. Train loss: 4.444929, Valid loss 4.555637, Time 00:01:54, lr 0.001
Epoch 8. Train loss: 4.408271, Valid loss 4.514169, Time 00:01:53, lr 0.001
Epoch 9. Train loss: 4.375457, Valid loss 4.606632, Time 00:01:54, lr 0.001
Epoch 10. Train loss: 4.340963, Valid loss 4.486368, Time 00:01:54, lr 0.001
Epoch 11. Train loss: 4.308109, Valid loss 4.474603, Time 00:01:54, lr 0.001
Epoch 12. Train loss: 4.267621, Valid loss 4.448845, Time 00:01:55, lr 0.001
Epoch 13.

In [15]:
import numpy as np

net = get_net(ctx)
net.hybridize()
train(net, train_valid_data, None, num_epochs, learning_rate, weight_decay,
      ctx, lr_period, lr_decay)

outputs = []
for data, label in test_data:
    output = nd.softmax(net(data.as_in_context(ctx)))
    outputs.extend(output.asnumpy())
ids = sorted(os.listdir(os.path.join(data_dir, input_dir, 'test/unknown')))
with open('submission.csv', 'w') as f:
    f.write('id,' + ','.join(train_valid_ds.synsets) + '\n')
    for i, output in zip(ids, outputs):
        f.write(i.split('.')[0] + ',' + ','.join(
            [str(num) for num in output]) + '\n')

Epoch 0. Train loss: 4.960529, Time 00:01:57, lr 0.001
Epoch 1. Train loss: 4.733697, Time 00:01:56, lr 0.001
Epoch 2. Train loss: 4.676125, Time 00:01:57, lr 0.001
Epoch 3. Train loss: 4.626623, Time 00:01:57, lr 0.001
Epoch 4. Train loss: 4.579331, Time 00:01:57, lr 0.001
Epoch 5. Train loss: 4.534541, Time 00:01:58, lr 0.001
Epoch 6. Train loss: 4.486066, Time 00:01:58, lr 0.001
Epoch 7. Train loss: 4.437353, Time 00:01:57, lr 0.001
Epoch 8. Train loss: 4.396183, Time 00:01:57, lr 0.001
Epoch 9. Train loss: 4.351972, Time 00:01:58, lr 0.001
Epoch 10. Train loss: 4.309416, Time 00:01:57, lr 0.001
Epoch 11. Train loss: 4.274161, Time 00:01:57, lr 0.001
Epoch 12. Train loss: 4.233147, Time 00:01:58, lr 0.001
Epoch 13. Train loss: 4.199011, Time 00:01:58, lr 0.001
Epoch 14. Train loss: 4.157985, Time 00:01:58, lr 0.001
Epoch 15. Train loss: 4.126619, Time 00:01:57, lr 0.001
Epoch 16. Train loss: 4.092342, Time 00:01:57, lr 0.001
Epoch 17. Train loss: 4.051833, Time 00:01:58, lr 0.001
Ep